In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [3]:
feed.summarize()

,table,column,num_values,num_nonnull_values,num_unique_values,min_value,max_value
0,agency,agency_name,1,1,1,Department of Transport and Main Roads - Trans...,Department of Transport and Main Roads - Trans...
1,agency,agency_url,1,1,1,http://www.sunbus.com.au,http://www.sunbus.com.au
2,agency,agency_timezone,1,1,1,Australia/Brisbane,Australia/Brisbane
3,agency,agency_lang,1,1,1,en,en
4,agency,agency_phone,1,1,1,(07)40576411,(07)40576411
0,calendar,service_id,4,4,4,CNS2014-CNS_MUL-Saturday-00,CNS2014-CNS_MUL-Weekday-00-0000100
1,calendar,monday,4,4,2,0,1
2,calendar,tuesday,4,4,2,0,1
3,calendar,wednesday,4,4,2,0,1
4,calendar,thursday,4,4,2,0,1


In [24]:
dates = feed.get_first_week()[6:]
feed1 = feed.restrict_to_dates(["20180101"])
feed1.routes


,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color


In [ ]:
f = gt.build_null_route_time_series(feed, dates[0], freq='12H', split_directions=True)
f

In [ ]:
#f = feed.compute_stop_time_series(dates[:3], split_directions=True, freq='12H')
#f = feed.compute_route_time_series(trip_stats, dates[:1], freq='12H', split_directions=True)
f = feed.compute_feed_time_series(trip_stats, dates[:1], freq='12H')
#f.columns
pd.isnull(f.values).all()

In [ ]:
ts = feed.compute_route_time_series(trip_stats, dates[0], freq='6H')
# inds = ['num_trips', 'num_trip_ends', 'num_trip_starts']
# rids = ['110-423', '111-423', '112-423']

# ts.loc[:, (inds, rids)]

#ts = feed.compute_feed_time_series(trip_stats, [dates[0], dates[2]], freq='6H')
# ts = feed.compute_stop_time_series([dates[0], dates[2]], freq='6H')
# sid = '750015'
# ts.loc[:, ('num_trips', sid)]

ts

In [ ]:
ts.columns

In [ ]:
dts = gt.downsample(ts, freq='D')
#dts.loc[:, (inds, rids)]
#dts.loc[:, ('num_trips', sid)]
dts

In [ ]:
f = feed.compute_feed_stats(trip_stats, '2001')
f


In [ ]:
f.date

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()